In [1]:
#Plaza Ayuntamiento @39.4697661,-0.4113992
#Church of San Juan del Hospital @39.4743981,-0.3814647
#Torres de los Serranos @39.4791963,-0.378187

In [20]:
import geopy.distance
import numpy as np
import pandas as pd
from datetime import datetime
from math import sin, cos, sqrt, atan2, radians

In [3]:
df = pd.read_csv('../vb_data/data.csv')

In [7]:
df.drop(['Unnamed: 0'], axis =1, inplace = True)

In [10]:
loc_names = {'PzaAyun-dist':[39.4697661,-0.4113992], 
             'SJChurch-dist':[39.4743981,-0.3814647], 
             'TorresS-dist':[39.4791963,-0.378187]}

In [21]:
for key, value in loc_names.items():
    a_list = []
    col_name = key
    lat1 = value[0]
    long1 = value[1]
    for i in range(0,len(df)):
        coords_1 = (lat1, long1)
        coords_2 = (df['Lat'][i], df['Long'][i])
        dist = geopy.distance.vincenty(coords_1, coords_2).km
        a_list.append(dist)
    df[col_name] = a_list

In [23]:
df['update'] = [datetime.strptime(x, '%d/%m/%Y %H:%M:%S') for x in df['update']]

In [31]:
df['update'][0].hour

5

In [29]:
df['crida'] = [1 if x.day == 25 else 0 for x in df['update']]

In [32]:
df['hour'] = [x.hour for x in df['update']]

In [33]:
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable, Independence, Autoregressive)
from statsmodels.genmod.families import Poisson

%matplotlib inline

In [44]:
df.rename(columns={'SJChurch-dist': 'SJChurch', 'PzaAyun-dist': 'PzaAyun', 'TorresS-dist':'Torres'}, inplace = True)

In [45]:
df.head()

,update,available,free,total,number,open,ticket,name,Long,Lat,SJChurch,PzaAyun,Torres,crida,hour
0,2018-02-20 05:27:07,0,24,25,4,T,T,004_PLAZA_DE_LA_VIRGEN_CALLE_BAILIA,-0.375341,39.476747,0.587944,3.198046,0.365928,0,5
1,2018-02-20 05:27:07,3,17,20,7,T,T,007_PZA_DEL_MERCADO_TAULA_DE_CANVIS,-0.379184,39.474872,0.203201,2.829435,0.487673,0,5
2,2018-02-20 05:27:07,14,0,15,168,T,F,168_AVDA. MALVARROSA,-0.327885,39.476871,4.618310,7.229214,4.335658,0,5
3,2018-02-20 05:27:07,0,16,16,229,T,F,229_CALLE_AITANA_ESQ_AVDA_BURJASSOT,-0.394805,39.493104,2.372876,2.958428,2.104336,0,5
4,2018-02-20 05:27:07,2,18,20,248,T,F,248_AVDA_TRES_CRUCES_JOSE_MARIA_MORTES_LERMA,-0.404968,39.462840,2.395227,0.947400,2.933968,0,5


In [62]:
df.to_csv('../vb_data/df_dist')

### We want our target variable to penalize being further away from .5 full because VB is optimized when each station can be (1) borrowed from *and* (2) docked at

In [54]:
df['target'] = (df['available']/df['total'])

In [56]:
df['target2'] = [x if x <= .5 else (.5-(x-.5)) for x in df['target']]

In [58]:
df['target2'] = 2*df['target2']

In [60]:
fam = Poisson()
m1 = GLM.from_formula('target2 ~ SJChurch + PzaAyun + Torres + crida + hour',  
                      df, family=fam)
r1 = m1.fit()
print(r1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                target2   No. Observations:               273000
Model:                            GLM   Df Residuals:                   272994
Model Family:                 Poisson   Df Model:                            5
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:            -1.9700e+05
Date:                Mon, 12 Mar 2018   Deviance:                       76300.
Time:                        18:22:10   Pearson chi2:                 6.19e+04
No. Iterations:                     5                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8646      0.010    -88.067      0.000      -0.884      -0.845
SJChurch       0.0116      0.011      1.066      0.2

In [48]:
df.drop_duplicates(inplace=True)

In [61]:
model_data = df[['SJChurch', 'PzaAyun', 'Torres', 'crida', 'hour']]
model_datac = sm.add_constant(model_data, prepend=False)

mod = sm.OLS(df['target2'], model_datac)

res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                target2   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     462.8
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        18:52:29   Log-Likelihood:                -69202.
No. Observations:              273000   AIC:                         1.384e+05
Df Residuals:                  272994   BIC:                         1.385e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SJChurch       0.0057      0.002      2.564      0.0